# Design space for IOD2

This design space estimation should give calculate a number of design parameters (boom cross section, boom length, etc.) from a given set of requirements (max power, max mass, minimum wall size, etc.).

The script is devided in the following parts:

- Input parameters
- Cross section data
- Extruder data
- Diagram methods


In [578]:
"""Imports"""

import solara  # type: ignore
import json
import matplotlib
import matplotlib.pyplot as plt  # type: ignore
import numpy as np  # type: ignore

from ipywidgets import interactive
from ipywidgets.widgets import FloatSlider, Checkbox, Dropdown, Button, VBox
from matplotlib.axes import Axes
from pydantic import BaseModel  # type: ignore
from math import pi, sqrt
from typing import Type, Union, List, Tuple, Any

### Input parameters

The input parameters are devided in:

- `StaticProperties`: physical properties and constants, parameters from hardware
- `RequirementParameters`: requirements from the launcher and the customer

#### StaticProperties

| **variable name**              | **describtion**                             | **per extr** |
| ------------------------------ | ------------------------------------------- | ------------ |
| al_density                     | density of the aluminum                     | n            |
| resin_density                  | density of the resin                        | n            |
| resin_viscosity                | viscosity resin                             | n            |
| resin_youngs_mod               | youngs modulus of the resin                 | n            |
| boom_alpha                     | first mod of the boom                       | n            |
| sf_resin_mass                  | safety factor for resin mass                | n            |
| feed_speed                     | feed speed of the array                     | n            |
| extrusion_pressure             | pressure difference at the nozzle           | y            |
| extruder_wall_thickness        | minimal wall thickness of the extruder      | y            |
| extruder_motor_torque_constant | torque constant of the extruder motor (F/I) | y            |
| extruder_motor_resistance      | resistance per winding                      | y            |
| extruder_motor_windings        | windings of the motor                       | y            |
| wall_size                      | minimal wall size of the boom               | n            |
| tube_diameter                  | tubing diameter                             | y            |
| tube_length                    | tubing length (from extr. to nozzle)        | y            |
| sheet_width                    | width of the array sheet                    | n            |
| mass_spreader                  | mass of the spreader bar                    | n            |
| mass_distributed_sheet         | mass per area of the sheet                  | n            |

#### RequirementParameters

| **variable name**    | **describtion**                       | **per extr** |
| -------------------- | ------------------------------------- | ------------ |
| deployment_frequency | boom deployment frequency requirement | n            |
| power                | maximal available power               | n            |
| mass                 | maximal available mass                | n            |
| extruder_diameter    | reference diameter                    | y            |

The classes `ExtruderOption` and `RequirementParameters` wrap the output for each datapoint for the visualization and the results.

In [579]:
class StaticProperties(BaseModel):
    al_density: float = 2.7  # g/cm^3
    resin_density: float = 1.8  # g/cm^3
    resin_viscosity: float = 70  # kPa * s
    resin_youngs_mod: float = 5000  # MPa

    boom_alpha: float = 1.875

    sf_resin_mass: float = 1.15

    feed_speed: float = 30  # mm/min
    extrusion_pressure: float = 80  # kPa
    extruder_wall_thickness: float = 2  # mm
    extruder_motor_torque_constant: float = 335  # N / A
    extruder_motor_resistance: float = 2  # Ohm
    extruder_motor_windings: float = 2  # -
    wall_size: float = 1  # mm

    tube_diameter: float = 6  # mm
    tube_length: float = 500  # mm

    sheet_width: float = 300  # mm
    mass_spreader: float = 0.1  # kg
    mass_distributed_sheet: float = 0.35  # kg/m^2


class RequirementParameters(BaseModel):
    deployment_frequency: float = 1.3  # Hz

    power: float = 16  # W
    mass: float = 1000  # g

    extruder_diameter: float = 55  ## mm


st_props = StaticProperties()
rq_params = RequirementParameters()
lengths: List[float] = np.arange(300, 2000, 1)

In [580]:
class ExtruderOption(BaseModel):
    diameter: float = 0  # mm
    length: float = 0  # mm
    pressure_difference: float = 0  # kPa
    mass_total: float = 0  # g
    actuator_force: float = 0  # N
    actuator_power: float = 0  # W


class DesignParameters(BaseModel):
    boom_length: float = 0  # mm

    cross_section: "CrossSection" = None  # type: ignore

    v_boom: float = 0  # mm^3
    v_total: float = 0  # mm^3
    flow_speed_tube: float = 0  # mm/min
    flow_volume: float = 0  # mm^3/min

    mass_resin: float = 0  # g

    extruder: ExtruderOption = None

### Cross section data

A `CrossSection` is a base class for all cross sections, which safes the `area`, `second_moment_of_inertia`, `wall_thickness`.
The following cross sections are implemented:

- `Tube` (`diameter`, `wall_thickness`)
- `IBeam` (`height`, `ratio`, `wall_thickness`)
- `Rectangle` (`height`, `wall_thickness`)

To calculate the frequency of the structure, use `cross_section.frequency()`, which is possible for all derived classes.

In [581]:
class CrossSection(BaseModel):
    type_name: str
    area: float  # mm^2
    sec_moment_of_area: float  # mm^4
    wall_thickness: float  # mm

    def frequency(self, length: float, st: StaticProperties):
        k = 3 * st.resin_youngs_mod * 10e6 * self.sec_moment_of_area / length**3
        mass_boom = st.resin_density * self.area * length / 1000  # kg
        mass_sheet_dis = st.mass_distributed_sheet * st.sheet_width / 2000
        mass_sheet = mass_sheet_dis * length / 1000  # kg
        mass = mass_sheet + mass_boom  # kg
        m_eff = 3 * mass / (st.boom_alpha**4) + 0.5 * st.mass_spreader
        return sqrt(k / (m_eff * 1000)) / (2 * pi)

    def factory(self, value: float) -> Union[NotImplementedError, "CrossSection"]:
        raise NotImplementedError("The 'factory' function is not implemented.")


class Tube(CrossSection, BaseModel):

    diameter: float

    def __init__(self, outer_diameter: float, wall_thickness: float) -> None:
        area = pi / 4 * (outer_diameter**2 - (outer_diameter - 2 * wall_thickness) ** 2)
        i = pi / 4 * (outer_diameter**4 - (outer_diameter - 2 * wall_thickness) ** 4)
        super().__init__(
            type_name="tube",
            area=area,
            sec_moment_of_area=i,
            wall_thickness=wall_thickness,
            diameter=outer_diameter,
        )

    def factory(self, value: float) -> "Tube":
        return Tube(value, self.wall_thickness)


class IBeam(CrossSection, BaseModel):

    ratio: float
    height: float

    def __init__(self, height: float, wall_thickness: float, ratio: float) -> None:
        w = height / ratio
        t = wall_thickness
        h = height - t
        area = 2 * w * t + h * t
        i = h**3 * w / 12 + 2 * (t**3 * w / 12 + t * w * (h + t) ** 2 / 4)
        super().__init__(
            type_name="ibeam",
            area=area,
            sec_moment_of_area=i,
            wall_thickness=wall_thickness,
            ratio=ratio,
            height=height,
        )

    def factory(self, value: float) -> "IBeam":
        return IBeam(value, self.wall_thickness, self.ratio)


class Rectangle(CrossSection, BaseModel):

    height: float

    def __init__(self, width: float, height: float) -> None:
        area = width * height
        i = width * height**3 / 12
        super().__init__(
            type_name="rectangle",
            area=area,
            sec_moment_of_area=i,
            wall_thickness=width,
            height=height,
        )

    def factory(self, value: float) -> "Rectangle":
        return Rectangle(self.wall_thickness, value)

### Extruder data

The function `get_extruder_data(DesignParameters, StaticProperties, RequirementParameters)` calculates the extruder properties (safed in `ExtruderOption`) based on the input values. 

In [582]:
def get_extruder_data(
    dp: DesignParameters, st: StaticProperties, rq: RequirementParameters
) -> ExtruderOption:

    d = rq.extruder_diameter

    l = (4 * dp.v_total) / (pi * d**2)

    # pressure estimation
    zeta = (0.707 * sqrt(1 - ((st.tube_diameter / d) ** 2))) ** 2
    flow_speed_extruder = 4 * dp.flow_volume / (d**2 * pi)
    pd_ext_1 = (
        8 * dp.flow_volume * st.resin_viscosity * l / (pi * (d / 2) ** 4 * 60 * 1000)
    )
    pd_ext_2 = 32 * st.resin_viscosity * l * flow_speed_extruder / (d**2 * 60000)
    pd_int = zeta * st.resin_density * dp.flow_speed_tube**2 / (2 * 3.6 * 10**9)
    pd_tube_1 = (8 * dp.flow_volume * st.resin_viscosity * st.tube_length) / (
        pi * (st.tube_diameter / 2) ** 4 * 60 * 1000
    )
    pd_tube_2 = (32 * st.resin_viscosity * st.tube_length * dp.flow_speed_tube) / (
        st.tube_diameter**2 * 60 * 1000
    )
    pd_total = pd_ext_1 + pd_ext_2 + pd_int + pd_tube_1 + pd_tube_2

    # mass estimation
    v_walls_1 = l * pi * ((d + 2 * st.extruder_wall_thickness) ** 2 - d**2) / 4
    v_walls_2 = st.extruder_wall_thickness * d**2 * pi / 4
    m_walls = (v_walls_1 + v_walls_2) * st.al_density / 1000

    # times two because of two extruders
    m_total = (dp.mass_resin + m_walls) * 2

    f_ac = (st.extrusion_pressure + pd_total) * pi * d**2 / 4000
    i_ac = f_ac / st.extruder_motor_torque_constant
    # times two because of two extruders
    p_ac = (i_ac**2 * st.extruder_motor_resistance * st.extruder_motor_windings) * 2

    return ExtruderOption(
        diameter=d,
        length=l,
        pressure_difference=pd_total,
        mass_total=m_total,
        actuator_force=f_ac,
        actuator_power=p_ac,
    )

### Diagram methods & UI elements

The following section wraps a bunch of UI functions and the solver function.
To solve the frequency requirement, a iterative solver iterates, till a solution fulfilles the requirement.

In [583]:
SOLVER_STEP_DELTA: float = 0.002


class DiagramClass(BaseModel):
    cs: CrossSection
    cs_p: float

    line: Union[str, Any]
    label: str

    solved: List[DesignParameters] = []


class UISetting(BaseModel):
    length: float
    boom: DiagramClass
    extruder_overview: bool

    dps: List[DesignParameters] = None
    dp: DesignParameters = None
    rq_params: RequirementParameters = None


def solver_iter(
    length: 2000,
    factory: Type[CrossSection],
    start: float,
    step: float,
) -> Tuple[CrossSection, float]:
    while True:
        m: CrossSection = factory.factory(start)
        if m.frequency(length, st_props) > rq_params.deployment_frequency:
            return [m, start]
        start += step


def solve_design_param(
    length: float,
    st: StaticProperties,
    rq: RequirementParameters,
    cs_init: CrossSection,
    cs_param: float,
) -> Tuple[DesignParameters, float]:

    dp = DesignParameters(boom_length=length)

    # cross section
    cs, cs_param = solver_iter(length, cs_init, cs_param, SOLVER_STEP_DELTA)
    dp.cross_section = cs

    # properties
    dp.v_boom = dp.cross_section.area * length
    v_tubing = st.tube_diameter**2 * pi * st.tube_length / 4
    dp.v_total = (dp.v_boom + v_tubing) * st.sf_resin_mass

    dp.mass_resin = dp.v_total * st.resin_density / 1000

    dp.flow_volume = dp.cross_section.area * st.feed_speed
    dp.flow_speed_tube = 4 * dp.flow_volume / (st.tube_diameter**2 * pi)

    # set extruder values
    dp.extruder = get_extruder_data(dp, st, rq)
    return dp, cs_param


def solve_design_params(
    length: List[float],
    st: StaticProperties,
    rq: RequirementParameters,
    cs_init: CrossSection,
    cs_param: float,
) -> List[DesignParameters]:

    dps: List[DesignParameters] = []

    for l in length:

        dp, cs_param = solve_design_param(l, st, rq, cs_init, cs_param)
        dps.append(dp)

    return dps


def stream(data: List[BaseModel], keys: List[Union[str, int]]) -> List[Any]:
    res: List[float] = []
    key = keys[0]
    if type(key) == str:
        for dp in data:
            res.append(getattr(dp, key, 0))
    else:
        for dp in data:
            res.append(dp[key])

    if len(keys) > 1:
        return stream(res, keys[1:])
    return res


def as_str(keys: List[Union[str, int]]) -> str:
    res = keys[0]
    for i in keys[1:]:
        res += "."
        res += str(i)
    return res


def plot_design_param(
    dcs: List[DiagramClass],
    keys: List[Union[str, int]],
    ax: Axes,
    clr: str,
    labels: Union[None, List[Tuple[str, str]]] = None,
):
    for dc in dcs:
        ax.plot(
            lengths,
            stream(dc.solved, keys),
            label=(dc.label + ": " + as_str(keys) if labels == None else ""),
            linestyle=dc.line,
            color=clr,
        )
    if labels is not None:
        for color, tag in labels:
            ax.plot([], [], color=color, label=tag)


def plot_custom_param(
    dps: List[DesignParameters],
    keys: List[Union[str, int]],
    ax: Axes,
    clr: str,
    label: Union[None, Tuple[str, str]] = None,
):
    ax.plot(
        lengths,
        stream(dps, keys),
        label=(as_str(keys) if label == None else label),
        color=clr,
    )


def plot_labels(ax: Axes, labels: List[Tuple[str, str]]):
    for clr, label in labels:
        ax.plot([], [], color=clr, label=label)


def plot_const(val: float, ax: Axes, clr: str, line: str = "-", label: str = ""):
    if label != "":
        ax.plot(
            [lengths[0], lengths[-1]],
            [val, val],
            color=clr,
            linestyle=line,
            label=label,
        )
    else:
        ax.plot([lengths[0], lengths[-1]], [val, val], color=clr, linestyle=line)


def plot_point(length: float, val: float, ax: Axes, clr: str):
    ax.scatter([length], [val], color=clr, marker="o", s=40, zorder=10)

In [584]:
wall = st_props.wall_size
cmap = plt.get_cmap("tab10")
c_current = "crimson"
c_max = "grey"

dclasses = [
    DiagramClass(cs=Tube(2.1, wall), cs_p=2.1, line="solid", label="Tube"),
    DiagramClass(cs=IBeam(2.1, wall, 2), cs_p=2.1, line=(0, (5, 10)), label="IBeam"),
    DiagramClass(cs=Rectangle(1, 5), cs_p=1, line=(0, (1, 10)), label="Rect"),
]

for dc in dclasses:
    dc.solved = solve_design_params(lengths, st_props, rq_params, dc.cs, dc.cs_p)

In [585]:
def get_json_result(dp: DesignParameters):
    if dp is None:
        return

    json_data = dp.model_dump_json()
    return json.dumps(
        json.JSONDecoder().decode(json_data),
        sort_keys=True,
        indent=4,
    )


def get_output_btn(ui_setting: "UISetting") -> solara.FileDownload:
    return solara.FileDownload.widget(
        data=lambda: get_json_result(ui_setting.dp),
        filename="output.json",
        label="Download file",
        mime_type="application/json",
    )

### Result


In [586]:
%matplotlib inline
matplotlib.rcParams["figure.dpi"] = 200

ui_sets = UISetting(length=lengths[0], boom=dclasses[0], extruder_overview=True)
ui_sets.rq_params = RequirementParameters(**rq_params.model_dump())
ui_sets.dp, _ = solve_design_param(ui_sets.length, st_props, ui_sets.rq_params, ui_sets.boom.cs, ui_sets.boom.cs_p)
ui_sets.dps = solve_design_params(lengths, st_props, ui_sets.rq_params, ui_sets.boom.cs, ui_sets.boom.cs_p)
ui_sets.dp.cross_section = ui_sets.boom.cs

def draw() -> None:
    """Redraws the diagram."""
    
    fig, ax = plt.subplots(3, 1, sharex=True)
    fig.set_size_inches(12, 8, forward=False)

    # ===== LABELS ===== 
    plt.xlabel("boom length [mm]")
    ax[0].set_ylabel("boom cross section \n [$mm^2$]")
    ax[1].set_ylabel("motor power \n [$W$]")
    ax[2].set_ylabel("total extr mass \n [$g$]")

    # ===== PLOT 0 ===== 
    plot_design_param(dclasses, ["cross_section", "area"], ax[0], cmap(0))
    plot_point(ui_sets.length, ui_sets.dp.cross_section.area, ax[0], c_current)

    # ===== PLOT 1 ===== 
    if (ui_sets.extruder_overview):
        plot_design_param(dclasses, ["extruder", "actuator_power"], ax[1], cmap(0), labels=[(cmap(0),f"Diameter = {rq_params.extruder_diameter}mm")])
    plot_custom_param(ui_sets.dps, ["extruder", "actuator_power"], ax[1], c_current, "Actual power")
    plot_const(rq_params.power, ax[1], c_max, "-", "Max power")
    plot_point(ui_sets.length, ui_sets.dp.extruder.actuator_power, ax[1], c_current)
    
    # ===== PLOT 2 ===== 
    if (ui_sets.extruder_overview):
        plot_design_param(dclasses, ["extruder", "mass_total"], ax[2], cmap(0), labels=[(cmap(0),f"Diameter = {rq_params.extruder_diameter}mm")])
    plot_custom_param(ui_sets.dps, ["extruder", "mass_total"], ax[2], c_current, "Actual mass")
    plot_const(rq_params.mass, ax[2], c_max, "-", "Max mass")
    plot_point(ui_sets.length, ui_sets.dp.extruder.mass_total, ax[2], c_current)

    # ===== LEGENDS ===== 
    for axis in ax:
        axis.grid()
        if len(axis.get_legend_handles_labels()[1]) > 0:
            axis.legend(prop={'size': 6}, loc='upper left')
    plt.show(block=False)

# GUI elements
beam_selector = Dropdown(options=[("Tube", 0),("IBeam", 1),("Rect", 2)], value=0, description="Boom: ")
length_slider = FloatSlider(value=lengths[0], min=lengths[0], max=lengths[-1], description="Length: ", step=0.5)
diameter_slider = FloatSlider(value=ui_sets.rq_params.extruder_diameter, min=20, max=100, description="Ex Diameter: ", step=0.5)
show_extruders = Checkbox(value=True, description="Show extruders")

def update(beam=beam_selector,length=length_slider, diameter=diameter_slider, extruder=show_extruders):
    ui_sets.length = length
    ui_sets.boom = dclasses[beam]
    ui_sets.rq_params.extruder_diameter = diameter
    ui_sets.extruder_overview = extruder

    # solve for given data
    ui_sets.dp, _ = solve_design_param(ui_sets.length, st_props, ui_sets.rq_params, ui_sets.boom.cs, ui_sets.boom.cs_p)
    ui_sets.dps = solve_design_params(lengths, st_props, ui_sets.rq_params, ui_sets.boom.cs, ui_sets.boom.cs_p)
    draw()
    
interactive_plot = interactive(update)
VBox([interactive_plot, get_output_btn(ui_sets)])